# A tool to relocate libraries, and fix evil relative paths

In [ ]:
import os
from subprocess import check_output as run

## Lib object

Stores the name of a library, its install name, its location and all its dependencies, as given by `otool -L`

In [ ]:
class Lib(object):
    location = None
    install_name = None
    name = None
    deps = None
    def __str__(self):
        mystr = ""
        mystr +=  "Name        :"+str(self.name)
        mystr +="\nInstall name:"+str(self.install_name)
        mystr +="\nLocation    :"+str(self.location)
        mystr +="\nDeps        : ... "
        return mystr

In [ ]:
# Walk the current tree, and extract all libraries
def get_libs():
    libraries = []
    basedir = run("pwd")

    for root, dirs, files in os.walk("./"):
        for f in files:
            filename = os.path.join(root, f)
            if os.path.isfile(filename) and f.endswith("dylib"):
                a = Lib()
                a.name = f
                a.install_name = run(["otool", "-D", filename]).split('\n')[1]
                a.location= basedir[0:-1]+filename[1::]
                long_deps = run(["otool", "-L", a.location]).split('\n')
                a.deps = [dep.partition(' ')[0][1::] for dep in long_deps[2:-1]]
                libraries += [a]
    return libraries

# Fix all install names first

Some will fail, because they are either stub files, or system files...

In [ ]:
libraries = get_libs()
c = 0
failed = []
for i in libraries:
    if os.path.islink(i.location):
        continue
    if i.install_name != i.location:
        try:
            run(["install_name_tool",'-id',i.location, i.location])
        except:
            print "Failed: ",i.name
            failed += [c]
    c += 1
print failed
print 'Removing failed libs...'
for fail in reversed(failed):
    del libraries[fail]

# Fix all dependencies with absolute paths

In [ ]:
c = 0
for this_lib in libraries:
    c += 1
    if this_lib.install_name != this_lib.location:
        print 'Not valid:', i.name
    else:
        lib = this_lib.name
        command = ['install_name_tool']
        for dep in this_lib.deps:
            for loc in libraries:
                if dep.find(loc.name) != -1 and dep != loc.install_name:
                    command += ['-change', dep, loc.install_name]
                    break
        try:
            if len(command) != 1:
                command += [this_lib.location]
                print 'Processing', lib
                print '======================\n\n'
                print " ".join(command)
                print '======================\n\n'
                run(command)
        except:
            print '\n\n*********************************************'
            print 'Last command failed!'
            print " ".join(command)
            print '*********************************************\n\n'